# 数据看板内容

## 平台上伦敦地区的总房源数、无评论房源数、各区房源数（listings_summary）

## 平台上的房东数、房东拥有的房源数的直方图（listings_summary）calculated_host_listings_count

## 随着时间变化的房价均值+中值折线图(Calendar)

## 随时间变化的房源新增率（listings_detail）

## 平台房源的总评分直方图

## 随时间变化的总评论数折线图（reviews_summary）

## RFM模型的饼图（RFM）

In [54]:
import numpy as np
import pandas as pd

# 数据筹备

## listings_summary

读取数据

In [24]:
listings_summary = pd.read_csv('../data/listings_summary.csv',
                          parse_dates=['last_review'])

listings_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85068 entries, 0 to 85067
Data columns (total 16 columns):
id                                85068 non-null int64
name                              85043 non-null object
host_id                           85068 non-null int64
host_name                         85056 non-null object
neighbourhood_group               0 non-null float64
neighbourhood                     85068 non-null object
latitude                          85068 non-null float64
longitude                         85068 non-null float64
room_type                         85068 non-null object
price                             85068 non-null int64
minimum_nights                    85068 non-null int64
number_of_reviews                 85068 non-null int64
last_review                       65062 non-null datetime64[ns]
reviews_per_month                 65062 non-null float64
calculated_host_listings_count    85068 non-null int64
availability_365                  85068 non-null 

选取会用到的特征

In [25]:
columns_1=['id','host_id','neighbourhood','number_of_reviews','calculated_host_listings_count']
listings_summary=listings_summary[columns_1]

listings_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85068 entries, 0 to 85067
Data columns (total 5 columns):
id                                85068 non-null int64
host_id                           85068 non-null int64
neighbourhood                     85068 non-null object
number_of_reviews                 85068 non-null int64
calculated_host_listings_count    85068 non-null int64
dtypes: int64(4), object(1)
memory usage: 3.2+ MB


选出非空置房源的数据，并记录相关空置房源数目

In [26]:
print('未开张房源有%d个'%(listings_summary[listings_summary.number_of_reviews==0].id.count()))
print('未开张房源的占比为:%f'%(int(listings_summary[listings_summary.number_of_reviews==0].id.count())/85068.0))

listings_summary=listings_summary[listings_summary['number_of_reviews']!=0]
listings_summary.reset_index(drop=True, inplace=True)

listings_summary.info()

未开张房源有20006个
未开张房源的占比为:0.235177
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65062 entries, 0 to 65061
Data columns (total 5 columns):
id                                65062 non-null int64
host_id                           65062 non-null int64
neighbourhood                     65062 non-null object
number_of_reviews                 65062 non-null int64
calculated_host_listings_count    65062 non-null int64
dtypes: int64(4), object(1)
memory usage: 2.5+ MB


## Calendar

In [55]:
Calendar = pd.read_csv('../对Calendar数据的处理/处理好异常值的data.csv',
                          parse_dates=['year_month'])

Calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31009989 entries, 0 to 31009988
Data columns (total 8 columns):
listing_id    float64
available     object
price         float64
year_month    datetime64[ns]
year          int64
month         int64
weekday       int64
season        object
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.8+ GB


选取所需特征

In [14]:
Calendar['listing_id']=Calendar['listing_id'].map(lambda x:-1 if str(x)=='nan' else int(x))
columns_2=['listing_id','price','year_month']
Calendar=Calendar[columns_2]

Calendar_1=Calendar.copy()

In [15]:
Calendar_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31009989 entries, 0 to 31009988
Data columns (total 3 columns):
listing_id    int64
price         float64
year_month    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 709.8 MB


根据listing_id+year_month聚合得到不同时期各个房源的price

In [16]:
price_means = Calendar_1['price'].groupby([Calendar_1['listing_id'],Calendar_1['year_month']]).mean()

Calendar_1 = pd.DataFrame(price_means)
Calendar_1.reset_index(inplace=True)
Calendar_1.head()

,listing_id,year_month,price
0,11551,2019-11-01,105.000000
1,11551,2019-12-01,111.935484
2,11551,2020-01-01,91.290323
3,11551,2020-02-01,90.000000
4,11551,2020-03-01,90.000000


In [17]:
price_median = Calendar['price'].groupby([Calendar['listing_id'],Calendar['year_month']]).median()
tt = pd.DataFrame(price_median)
tt.reset_index(inplace=True)
Calendar_1['price_median']=tt['price']

In [20]:
Calendar_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104557 entries, 0 to 1104556
Data columns (total 4 columns):
listing_id      1104557 non-null int64
year_month      1104557 non-null datetime64[ns]
price           1104557 non-null float64
price_median    1104557 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 33.7 MB


In [23]:
Calendar_1.head()

,listing_id,year_month,price,price_median
0,11551,2019-11-01,105.000000,105.0
1,11551,2019-12-01,111.935484,110.0
2,11551,2020-01-01,91.290323,90.0
3,11551,2020-02-01,90.000000,90.0
4,11551,2020-03-01,90.000000,90.0


## listings_detail

In [27]:
listings_detail = pd.read_csv('../data/listings.csv', low_memory=False,
                                 parse_dates=['host_since', 
                                              'last_scraped', 'calendar_last_scraped',
                                              'first_review', 'last_review'])

这里用房源的第一条评论时间作为房源的开张时间

In [28]:
columns_3=['id','first_review']
scores = ['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
               'review_scores_communication', 'review_scores_location', 'review_scores_value']
listings_detail=listings_detail[columns_3+scores]

listings_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85068 entries, 0 to 85067
Data columns (total 8 columns):
id                             85068 non-null int64
first_review                   65062 non-null datetime64[ns]
review_scores_accuracy         63393 non-null float64
review_scores_cleanliness      63407 non-null float64
review_scores_checkin          63345 non-null float64
review_scores_communication    63401 non-null float64
review_scores_location         63347 non-null float64
review_scores_value            63346 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 5.2 MB


提取出日期的年份出来

In [29]:
listings_detail['first_review_year']=listings_detail['first_review'].dt.year

计算总平均分

In [30]:
listings_detail['score']=( listings_detail['review_scores_accuracy']+listings_detail['review_scores_cleanliness']
                               +listings_detail['review_scores_checkin']+listings_detail['review_scores_communication']
                               +listings_detail['review_scores_value']+listings_detail['review_scores_location'])/6

In [31]:
listings_detail=listings_detail[['id','first_review_year','score']]
listings_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85068 entries, 0 to 85067
Data columns (total 3 columns):
id                   85068 non-null int64
first_review_year    65062 non-null float64
score                63323 non-null float64
dtypes: float64(2), int64(1)
memory usage: 1.9 MB


In [32]:
listings_detail.head(3)

,id,first_review_year,score
0,11551,2010.0,9.333333
1,38151,NaN,NaN
2,13913,2010.0,9.666667


## reviews_summary

In [33]:
reviews_summary = pd.read_csv('../data/reviews_summary.csv', parse_dates=['date'])

reviews_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1486236 entries, 0 to 1486235
Data columns (total 2 columns):
listing_id    1486236 non-null int64
date          1486236 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 22.7 MB


In [34]:
reviews_summary.head()

,listing_id,date
0,11551,2010-03-21
1,38407,2010-09-22
2,38407,2010-11-02
3,38407,2010-11-09
4,38407,2010-11-13


提取出年份，根据年份聚合评论数目

In [35]:
reviews_summary['year']=reviews_summary['date'].dt.year

In [36]:
reviews_counts = reviews_summary['listing_id'].groupby([reviews_summary['year']]).count()

reviews_counts = pd.DataFrame(reviews_counts)
reviews_counts.reset_index(inplace=True)
reviews_counts.head()

,year,listing_id
0,2009,1
1,2010,196
2,2011,1457
3,2012,6296
4,2013,18094


删除2009年数据

In [37]:
reviews_counts=reviews_counts[reviews_counts.year!=2009]

reviews_counts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 1 to 10
Data columns (total 2 columns):
year          10 non-null int64
listing_id    10 non-null int64
dtypes: int64(2)
memory usage: 240.0 bytes


# 连接listings_summary和listings_detail文件

In [38]:
listings_summary.head(2)

,id,host_id,neighbourhood,number_of_reviews,calculated_host_listings_count
0,11551,43039,Lambeth,185,2
1,13913,54730,Islington,19,2


In [39]:
listings_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65062 entries, 0 to 65061
Data columns (total 5 columns):
id                                65062 non-null int64
host_id                           65062 non-null int64
neighbourhood                     65062 non-null object
number_of_reviews                 65062 non-null int64
calculated_host_listings_count    65062 non-null int64
dtypes: int64(4), object(1)
memory usage: 2.5+ MB


In [40]:
listings_detail.head(2)

,id,first_review_year,score
0,11551,2010.0,9.333333
1,38151,NaN,NaN


In [41]:
listings_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85068 entries, 0 to 85067
Data columns (total 3 columns):
id                   85068 non-null int64
first_review_year    65062 non-null float64
score                63323 non-null float64
dtypes: float64(2), int64(1)
memory usage: 1.9 MB


In [42]:
listing=pd.merge(listings_summary,listings_detail) # 列名不同，但内容有相同，所以依然可以作为键来用
listing.head(3)

,id,host_id,neighbourhood,number_of_reviews,calculated_host_listings_count,first_review_year,score
0,11551,43039,Lambeth,185,2,2010.0,9.333333
1,13913,54730,Islington,19,2,2010.0,9.666667
2,90700,491286,Kensington and Chelsea,339,2,2011.0,9.500000


In [43]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65062 entries, 0 to 65061
Data columns (total 7 columns):
id                                65062 non-null int64
host_id                           65062 non-null int64
neighbourhood                     65062 non-null object
number_of_reviews                 65062 non-null int64
calculated_host_listings_count    65062 non-null int64
first_review_year                 65062 non-null float64
score                             63321 non-null float64
dtypes: float64(2), int64(4), object(1)
memory usage: 4.0+ MB


检查缺失值

In [44]:
listing.isnull().sum()

id                                   0
host_id                              0
neighbourhood                        0
number_of_reviews                    0
calculated_host_listings_count       0
first_review_year                    0
score                             1741
dtype: int64

均值填充

In [45]:
mean_val = listing['score'].mean()
listing['score'].fillna(mean_val, inplace=True)

# 筛选Calendar的数据

In [46]:
Calendar_2=Calendar_1.copy()

In [48]:
temp=listing['id'].drop_duplicates().values.tolist()

Calendar_2['flag']=Calendar_2['listing_id'].map(lambda x:1 if x in temp else 0)

In [49]:
Calendar_2=Calendar_2[Calendar_2.flag==1]
Calendar_2=Calendar_2.drop('flag',axis=1)

In [52]:
Calendar_2.head()

,listing_id,year_month,price,price_median
0,11551,2019-11-01,105.000000,105.0
1,11551,2019-12-01,111.935484,110.0
2,11551,2020-01-01,91.290323,90.0
3,11551,2020-02-01,90.000000,90.0
4,11551,2020-03-01,90.000000,90.0


筛选完数据后再次根据日期聚合

In [50]:
price_means = Calendar_2['price'].groupby([Calendar_2['year_month']]).mean()

Calendar_3 = pd.DataFrame(price_means)
Calendar_3.reset_index(inplace=True)
Calendar_3.head()

,year_month,price
0,2019-11-01,104.655354
1,2019-12-01,113.599907
2,2020-01-01,108.298320
3,2020-02-01,107.408484
4,2020-03-01,109.906569


In [51]:
price_median = Calendar_2['price'].groupby([Calendar_2['year_month']]).median()

Calendar_4 = pd.DataFrame(price_means)
Calendar_4.reset_index(inplace=True)

Calendar_3['price_median']=Calendar_4['price']
Calendar_3.head()

,year_month,price,price_median
0,2019-11-01,104.655354,104.655354
1,2019-12-01,113.599907,113.599907
2,2020-01-01,108.298320,108.298320
3,2020-02-01,107.408484,107.408484
4,2020-03-01,109.906569,109.906569


# 输出数据文件

In [103]:
Calendar_1.to_csv('Calendar.csv',index=0)

listing.to_csv('listing.csv',index=0)

reviews_summary.to_csv('reviews_summary.csv',index=0)